In [2]:
import sys
sys.path.append("../py/")

import numpy as np
import subprocess
import importlib
import mlflow as ml

import _conditions1 as cn
import _functions1 as fn
import _sim_iter1 as si

from multiprocessing import Manager
from bart_survival import surv_bart as sb



WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [41]:
manager = Manager()
l = manager.list()

In [34]:
del l

In [35]:
manager.shutdown()

In [36]:
import multiprocessing as mp

In [5]:
SPLIT_RULES1 =  [
    "pmb.ContinuousSplitRule()", 
]
model_dict1 = {"trees": 10,
    "split_rules": SPLIT_RULES1
}
sampler_dict1 = {
            "draws": 50,
            "tune": 10,
            "cores": 7,
            "chains": 7,
            "compute_convergence_checks": False
        }


In [1]:

    
    
# for i in range(1):
    
#     pb_sv = fn.get_py_bart_surv(x_mat, event_dict, model_dict1, sampler_dict1)
    


In [46]:
rng = np.random.default_rng(99)
scenario, x_mat, event_dict, sv_true, sv_scale_true = fn.get_sim(rng, 100, **cn.simple_1_1)
y_sk = sb.get_y_sklearn(status = event_dict["status"], t_event=event_dict["t_event"])
trn = sb.get_surv_pre_train(y_sk = y_sk, x_sk = x_mat, weight=None)
post_test = sb.get_posterior_test(y_sk = y_sk, x_test = x_mat)
small_post_x = np.unique(post_test["post_x"][:,0]).reshape(-1,1)
small_coords = np.hstack([np.repeat(0, small_post_x.shape[0]), np.repeat(1, small_post_x.shape[0]), ])
small_post_x = np.vstack([small_post_x, small_post_x])

BSM = sb.BartSurvModel(model_config=model_dict1, sampler_config=sampler_dict1)

# fit with just the time column
BSM.fit(y=trn["y"], X=trn["x"][:,0].reshape(-1,1), weights=trn["w"], coords = trn["coord"], random_seed=99)

# post1 = BSM.sample_posterior_predictive(X_pred=small_post_x, coords=small_coords)

mean shape 0.7999999999999998
mean scale 2.5
mean time draws 2.266307558869482
cens mean 9.44
(249, 1)


Only 50 samples in chain.
Multiprocess sampling (7 chains in 7 jobs)
PGBART: [f]


Sampling 7 chains for 10 tune and 50 draw iterations (70 + 350 draws total) took 5 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data
	> predictor_names
	> offset

In [50]:
x = mp.active_children()
x[0].kill()